[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ktehranchi/energy310_notebooks/blob/main/notebooks/05_configuring_pypsa_usa.ipynb)


# Configuring PyPSA-USA

This notebook demonstrates how to modify the `config.pypsa_intro.yaml` file to create different electricity sector networks. We'll cover:

1. **Multi-horizon networks** - Building models with multiple planning periods
2. **Temporal resampling** - Reducing temporal resolution for faster computation
3. **Using Opts** - Applying optional constraints and features
4. **Spatial Configurations** - Configuring geographical scope and network topology

All examples focus on electricity-only networks (not sector-coupled).

## 1. Temporal Horizon Configuration


### Multi-Horizon Configuration

To create a multi-horizon network, specify multiple years in `planning_horizons` and choose a `foresight` option:

```yaml
scenario:
  planning_horizons: [2030, 2040, 2050]  # Multiple planning horizons
  # ... other scenario settings

foresight: 'perfect'  # or 'myopic'
```

### Foresight Options

- **`perfect`**: Creates a monolithic optimization model where all planning horizons are optimized simultaneously. Future horizon values of costs and demand are incorporated into decisions made in earlier planning horizons.
- **`myopic`**: Solves each planning horizon sequentially, passing results forward to subsequent periods.

### Example: Multi-Horizon Configuration

```yaml
# Multi-horizon with perfect foresight
scenario:
  interconnect: [western]
  clusters: [4a]
  simpl: [50]
  opts: [RPS-REM-TCT-4h]
  ll: [v1.0]
  scope: "total"
  sector: ""  # Electricity only
  planning_horizons: [2030, 2040, 2050]  # Three planning periods

foresight: 'perfect'  # All periods optimized together
```

### Example: Myopic Multi-Horizon

```yaml
# Multi-horizon with myopic foresight
scenario:
  planning_horizons: [2030, 2040, 2050]

foresight: 'myopic'  # Sequential optimization
```

### Demand Data Configuration

The choice of demand data source is critical and **must align with your selected planning horizons**. PyPSA-USA offers two demand data sources:

#### Historical Demand (EIA)

Historical demand data is sourced from **EIA930** via the GridEmissions tool:
- **Available years**: 2018-2023
- **Data resolution**: Balancing area region level
- **Use case**: Historical simulations and back-casting

**Configuration for historical demand:**

```yaml
scenario:
  planning_horizons: [2019, 2020, 2021, 2022]  # Must be within 2018-2023

electricity:
  demand:
    profile: eia  # Use EIA for historical data
```

#### Forecasted Demand (EFS)

Forecasted demand is sourced from the **NREL Electrification Futures Study (EFS)**:
- **Available years**: 2018, 2020, 2024, 2030, 2040, 2050
- **Data resolution**: State-level hourly demand forecasts
- **Use case**: Forward-looking planning studies
- **Features**: Includes varying levels and speeds of electrification

**Configuration for forecasted demand:**

```yaml
scenario:
  planning_horizons: [2030, 2040, 2050]  # Must be betweet 2024 - 2050

electricity:
  demand:
    profile: efs  # Use EFS for forecasted data
    scenario:
      efs_case: reference  # Options: reference, medium, high
      efs_speed: moderate  # Options: slow, moderate, rapid
      aeo: reference  # AEO scenario for scaling
```

#### EFS Scenario Options

The EFS data includes different electrification scenarios:

- **`efs_case`**: Level of electrification adoption
  - `reference`: Reference case
  - `medium`: Medium electrification
  - `high`: High electrification

- **`efs_speed`**: Speed of electrification
  - `slow`: Slow electrification pace
  - `moderate`: Moderate electrification pace
  - `rapid`: Rapid electrification pace

#### Important: Planning Horizon Alignment

**The demand profile choice must match your planning horizons:**

- **For `profile: eia`**: Planning horizons must be within **2018-2023**
- **For `profile: efs`**: Planning horizons must be one of **2018, 2020, 2024, 2030, 2040, 2050**

If you select planning horizons that don't align with the chosen demand profile, PyPSA-USA will raise an error during the build process.

#### Example: Multi-Horizon with EFS Demand

```yaml
scenario:
  interconnect: [western]
  clusters: [4a]
  simpl: [50]
  opts: [RPS-REM-TCT-4h]
  ll: [v1.0]
  scope: "total"
  sector: ""  # Electricity only
  planning_horizons: [2030, 2040, 2050]  # EFS-supported years

foresight: 'perfect'

electricity:
  demand:
    profile: efs  # Must use EFS for 2030, 2040, 2050
    scenario:
      efs_case: reference
      efs_speed: moderate
      aeo: reference
```

#### Example: Historical Simulation with EIA Demand

```yaml
scenario:
  planning_horizons: [2020, 2021, 2022]  # Historical years

electricity:
  demand:
    profile: eia  # Must use EIA for historical years
```

**Note**: For years between the available EFS years (e.g., 2025, 2035), PyPSA-USA implements a scaling factor that interpolates between available years or scales historical demand using forecasts from the Annual Energy Outlook (AEO).

## 2. Temporal Resampling


Temporal resampling reduces the time resolution of your model, which can significantly speed up computation while maintaining reasonable accuracy. There are two main approaches:

### Using Opts Wildcard (Hourly Averaging)

The `opts` wildcard supports temporal resampling by averaging over every `n` hours. Use `nH` where `n` is the number of hours (e.g., `2H`, `3H`, `4H`, `6H`).

```yaml
scenario:
  opts: [RPS-REM-TCT-4h]  # 4h means average every 4 hours
```

This will resample the time-resolution by averaging over every 4 snapshots, effectively reducing a 8760-hour year to 2190 time steps.

### Example Configurations

```yaml
# 2-hour averaging (4380 time steps per year)
scenario:
  opts: [RPS-REM-TCT-2h]

# 3-hour averaging (2920 time steps per year)
scenario:
  opts: [RPS-REM-TCT-3h]

# 4-hour averaging (2190 time steps per year)
scenario:
  opts: [RPS-REM-TCT-4h]
```


**Note**: Temporal resampling is applied in the `prepare_network` step and can significantly reduce solve times while maintaining solution quality for many use cases.


## 3. Using Opts (Policy Constraints)



The `opts` wildcard triggers constraints and features in PyPSA-USA. Multiple options can be combined using hyphens (e.g., `RPS-REM-TCT-4h`).

### Available Opts

| Opt | Description | Applied In |
|-----|-------------|------------|
| `nH` (e.g., `2H`, `4H`, `6H`) | Resample time-resolution by averaging over every `n` snapshots | `prepare_network` |
| `nSEG` (e.g., `4380SEG`) | Apply time series segmentation to `n` adjacent snapshots | `prepare_network` |
| `REM` | Add regional CO₂ emissions limits | `solve_network` |
| `RPS` | Add Renewable Portfolio Standard constraints | `solve_network` |
| `PRM` | Add Regional Planning Reserve Margin constraints | `solve_network` |
| `ERM` | Add Regional Energy Reserve Margin constraints | `solve_network` |
| `TCT` | Add technology capacity targets (min/max per carrier/region/horizon) | `solve_network` |

### Example: Regional Emissions Limits (REM)

```yaml
scenario:
  opts: [REM]

electricity:
  regional_Co2_limits: 'config/CH1/policy_constraints/regional_Co2_limits_zero.csv'
```

The REM option adds regional carbon-dioxide emissions limits. Limits can be specified for:
- ReEDS zones (e.g., `p8`, `p9`)
- States (e.g., `CA`, `TX`)
- Interconnects (e.g., `western`, `eastern`, `texas`, `usa`)
- NERC regions
- 'all' will apply to the entire model

### Example: Renewable Portfolio Standard (RPS)

```yaml
scenario:
  opts: [RPS]

electricity:
  portfolio_standards: 'config/CH1/policy_constraints/portfolio_standards.csv'
```

RPS constraints set minimum shares of generator production (MWh) per carrier for individual regions. Standards can be set for single carriers or groups of carriers, and can be designated for specific planning horizons in multi-period models.

### Example: Technology Capacity Targets (TCT)

```yaml
scenario:
  opts: [TCT]

electricity:
  technology_capacity_targets: 'config/CH1/policy_constraints/technology_capacity_targets.csv'
```

TCT adds minimum or maximum levels of generator nominal capacity per carrier(s), per region(s), per investment horizon.

**Warning**: TCT targets can only be used with renewable generators and utility scale batteries in sector studies.

### Example: Emission Prices (Ep)

```yaml
scenario:
  opts: [Ep]

costs:
  emission_prices:
    enable: true
    co2: 50.0  # $/tonne CO₂
    co2_monthly_prices: false
```

This adds CO₂ emission prices to the marginal cost of generators.

### Example: Combined Opts

```yaml
scenario:
  opts: [RPS-REM-TCT-4h]  # RPS + REM + TCT + 4-hour temporal resampling
```

This configuration combines:
- Renewable Portfolio Standards
- Regional Emissions Limits
- Technology Capacity Targets
- 4-hour temporal averaging


## 4. Spatial Configurations



Spatial configuration determines the geographical scope and network topology of your model. This includes selecting interconnects, transmission networks, and filtering specific regions.

### 4.1 Interconnect Selection

The `interconnect` wildcard sets the geographic scope of the model:

```yaml
scenario:
  interconnect: [western]  # Options: western, eastern, texas, or usa
```

- **`western`**: Western Interconnection
- **`eastern`**: Eastern Interconnection  
- **`texas`**: ERCOT (Texas)
- **`usa`**: Entire United States (all three interconnections)

### 4.2 Transmission Network Selection

Choose between two transmission network topologies:

```yaml
model_topology:
  transmission_network: 'reeds'  # Options: 'reeds' or 'tamu'
  topological_boundaries: 'reeds_zone'  # Options: 'county' or 'reeds_zone'
```

- **`reeds`**: The ReEDS NARIS networks (recommended for most use cases)
- **`tamu`**: The synthetic BE-TAMU nodal network

For ReEDS networks, you must also specify `topological_boundaries`:
- **`county`**: County-level resolution
- **`reeds_zone`**: ReEDS zone resolution

### 4.3 Filtering Regions

You can filter the model to include only specific regions using the `include` option:

#### Example: Modeling California Only

```yaml
scenario:
  interconnect: [western]  # or [usa]

model_topology:
  transmission_network: 'reeds'
  topological_boundaries: 'reeds_zone'
  include:
    reeds_state: ['CA']  # Include entire state of California
```

Alternatively, you can specify specific ReEDS zones:

```yaml
model_topology:
  include:
    reeds_zone: ['p8', 'p9', 'p10', 'p11']  # California ReEDS zones
```

#### Example: Filtering by Balancing Authority

```yaml
model_topology:
  include:
    reeds_ba: ['ERCO']  # ERCOT balancing authority
```

#### Example: Filtering by Transmission Region

```yaml
model_topology:
  include:
    trans_reg: ['region1', 'region2']  # Specific transmission regions
```

**Note**: Mixed zone types are not supported in the `include` section.

### 4.4 Network Resolution

Control the number of nodes in your network using `clusters` and `simpl`:

```yaml
scenario:
  clusters: [4a]  # Final number of transmission nodes
  simpl: [50]     # Number of nodes after initial simplification
```

#### Resource Resolution vs Transmission Resolution

You can independently configure resource zones from the transmission network:

```yaml
scenario:
  clusters: [10m]  # 10 transmission nodes (m = mixed, keeps resource detail)
  simpl: [100]     # 100 distinct renewable resource zones
```

The `m` suffix after `clusters` results in:
- 10 transmission nodes
- 100 distinct renewable resource zones

Other suffixes:
- **`c`**: All conventional resources from `simpl` step will not be clustered
- **`a`**: All resources will not be clustered beyond the `simpl` level

### 4.5 Aggregating Regions

You can aggregate (copper-plate) specific regions using the `aggregate` option:

#### Example: Meshed ReEDS NARIS WECC Topology

```yaml
scenario:
  interconnect: [western]
  clusters: [87]
  simpl: [380]

model_topology:
  transmission_network: 'reeds'
  topological_boundaries: 'county'
  include:
    # Nothing specified - modeling entire WECC
  aggregate:
    trans_grp: ['NorthernGrid_South', 'NorthernGrid_West', 'NorthernGrid_East', 
                'WestConnect_North', 'WestConnect_South']
```

This configuration:
- Models California at county-level resolution
- Copper-plates (aggregates) Non-CA WECC regions listed under `trans_grp`
- Creates a copper-plate network where resources can be clustered and shared across aggregated regions

#### Example: Aggregating ReEDS Zones

```yaml
model_topology:
  aggregate:
    reeds_zone: ['p8', 'p9', 'p10', 'p11']  # Aggregate these zones
```

**Note**: For custom aggregation, you'll need information on region memberships from `workflow/repo_data/ReEDS_Constraints/membership.csv`.

### 4.6 Complete Spatial Configuration Example

```yaml
scenario:
  interconnect: [western]
  clusters: [4a]
  simpl: [50]
  opts: [RPS-REM-TCT-4h]
  ll: [v1.0]
  scope: "total"  # Options: "urban", "rural", or "total"
  sector: ""      # Empty for electricity-only

model_topology:
  transmission_network: 'reeds'
  topological_boundaries: 'reeds_zone'
  interface_transmission_limits: false
  include:
    reeds_state: ['CA']  # California only
  # aggregate:  # Uncomment to aggregate regions
  #   trans_grp: ['region1', 'region2']
```

### Minimum Node Requirements

Each clustering and interconnection option has a different minimum number of nodes. If you select a value too low, an error will be thrown in `cluster_network` notifying you of the correct minimum number. Refer to the transmission nodes table in the documentation for specific requirements.
